# Time Decay
**Truth Premise: When a day ends and a new day begins in life of options premium, its value depreciates because of time decay because it gets closer to expiry and the probability of its contract going as expected by the buyer gets lesser.**

## Strategy
1. Short both put and call at day closing right before the expiry
2. Square of in the morning

## Pseudocode
```python
nifty_at_0320
at_the_money_call_strike
at_the_money_put_strike
call_strike_premium_at_0320
put_strike_premium_at_0320
call_strike_premium_at_0915
put_strike_premium_at_0915
call_strike_premium_at_0916
put_strike_premium_at_0916
call_strike_premium_at_0917
put_strike_premium_at_0917
call_strike_premium_at_0918
put_strike_premium_at_0918
pnl_0915
pnl_0916
pnl_0917
pnl_0918
```

In [1]:
import datetime as dt
import utils as ut
import pandas as pd
import icharts as ic
from functools import cache
from constants import *


TEST_START = dt.datetime.strptime("2021-01-01", "%Y-%m-%d")
TEST_END = dt.datetime.strptime("2023-12-31", "%Y-%m-%d")
SYMBOL = "NIFTY 50"
IC_SYMBOL = "NIFTY"
INTERVAL = ut.INTERVAL_MIN1
EXCHANGE = ut.EXCHANGE_NSE
DAILY_INVESTMENT = 1 * (10 ** 5) # 1 lakh
LOT_SIZE = 50
LOT_QTY = 5

pd.set_option("display.max_colwidth", None)
pd.set_option("display.max_rows", 200)
# pd.set_option('precision', 2)
pd.set_option("display.precision", 2)
# pd.set_option('display.float_format', lambda x: '%.2f' % x)

def build_date_range(date_start, date_end, symbol):
    date_range = []
    cur_date = date_start
    while cur_date < date_end:
        if cur_date.weekday() not in [5, 6]:
            has_data, _ = ut.has_data(symbol, cur_date, interval=INTERVAL, exchange=EXCHANGE)
            if has_data:
                date_range.append(cur_date)
        cur_date += dt.timedelta(days=1)
    return date_range

all_dates = pd.DataFrame({"trade_date": build_date_range(TEST_START, TEST_END, SYMBOL)})
all_dates_shuffled = all_dates.sample(frac=1, random_state=42)

train_size = int(0.5 * len(all_dates_shuffled))
train_dates = all_dates_shuffled.iloc[:train_size]
test_dates = all_dates_shuffled.iloc[train_size:]
train_dates = all_dates
train_dates = train_dates.sort_values(by="trade_date")
train_dates.set_index("trade_date", inplace=True)
test_dates = test_dates.sort_values(by="trade_date")
test_dates.set_index("trade_date", inplace=True)

@cache
def get_intraday_data(date):
    return ut.get_data(symbol=SYMBOL, date=date, interval=INTERVAL, exchange=EXCHANGE)

@cache
def get_symbol_first_candle(symbol, trade_date):
    data = ut.get_data(symbol=SYMBOL, date=trade_date, interval=INTERVAL, exchange=EXCHANGE)
    return data.iloc[0].open, data.iloc[0].high, data.iloc[0].low, data.iloc[0].close

@cache
def get_first_candle_close(symbol, trade_date):
    data = ut.get_data(symbol=SYMBOL, date=trade_date, interval=INTERVAL, exchange=EXCHANGE)
    return data.iloc[0].close

@cache
def get_last_trading_day(date):
    return ut.get_last_trading_day(SYMBOL, date, interval=INTERVAL, exchange=ut.EXCHANGE_NSE)

train_dates["previous_trading_day"] = None
train_dates["previous_trading_day"] = train_dates.apply(lambda row: get_last_trading_day(row.name), axis=1)
train_dates["expiry"] = train_dates.apply(lambda row: ut.find_nclosest_expiry(SYMBOL, row.name, 1), axis=1)

def get_nifty_price(d, t):
    data = ut.get_data(symbol=SYMBOL, date=d, interval=INTERVAL, exchange=EXCHANGE)
    try:
        return data.loc[data.index.time == t].iloc[0].open
    except IndexError:
        return pd.NA
    except AttributeError as e:
        return pd.NA

buy_time = dt.time(hour=15, minute=28)
sell_time = dt.time(hour=9, minute=18)

train_dates = train_dates.copy()

# train_dates["trade_day_before_expiry"] = train_dates.expiry - pd.Timedelta(days=day_before_expiry)
train_dates.loc[:, "nifty_at_0320"] = train_dates.apply(lambda r: get_nifty_price(r.previous_trading_day.date(), dt.time(hour=15, minute=20)), axis=1)
train_dates.loc[:, "nifty_at_buy"] = train_dates.apply(lambda r: get_nifty_price(r.previous_trading_day.date(), buy_time), axis=1)
train_dates.loc[:, "nifty_at_sell"] = train_dates.apply(lambda r: get_nifty_price(r.name.date(), sell_time), axis=1)
train_dates.loc[:, "nifty_diff"] = train_dates.nifty_at_sell - train_dates.nifty_at_buy

train_dates = train_dates.loc[train_dates.nifty_at_0320.notna()]
train_dates["atm_strike"] = train_dates.apply(lambda trade: round(trade.nifty_at_0320 / 50) * 50, axis=1)

In [2]:
def get_premium_df(trade, strike_price, td, option_type, tm):
    pr = ic.get_opt_pre_df(symbol=IC_SYMBOL, expiry=trade.expiry, cur_dt=td, strike_price=strike_price, option_type=option_type)
    if type(pr) == type(pd.NA) or pr.shape[0] == 0:
        return pd.NA
    x = pr.loc[(pr.index.date == td) & (pr.index.time >= tm)]
    # # print(x.iloc[0])
    if x.shape[0] == 0:
        # print(tm)
        # print(td)
        # print(pr.loc[(pr.index.date == td)])
        # print(trade)
        return pd.NA
    return x.iloc[0].close

def get_next_1000(strike, i):
    divider = 100
    reminder = strike % divider
    if reminder != 0:
        return divider * (strike // divider) - i * divider
    return divider * (strike // divider) - (i+1) * divider

last_strike = None
"""
Results Training dataset 2nd expiry
    Call PnL at 9:18 for strike: -10, Total: -759835.0, Per Day: -2140.3802816901407, Days: 355, Pc Mean: -2.14975530819984
Call PnL at 9:18 for strike: -9, Total: -312092.4999999999, Per Day: -857.3969780219777, Days: 364, Pc Mean: -0.8748627952745024
Call PnL at 9:18 for strike: -8, Total: 71502.49999999994, Per Day: 195.36202185792334, Days: 366, Pc Mean: 0.18297912502349303
Call PnL at 9:18 for strike: -7, Total: 506540.0, Per Day: 1380.2179836512262, Days: 367, Pc Mean: 1.369693395280383
Call PnL at 9:18 for strike: -6, Total: 641875.0, Per Day: 1744.2255434782608, Days: 368, Pc Mean: 1.743611133234208
Call PnL at 9:18 for strike: -5, Total: 755737.5, Per Day: 2053.634510869565, Days: 368, Pc Mean: 2.054257620026519
    Call PnL at 9:18 for strike: -4, Total: 826650.0, Per Day: 2246.3315217391305, Days: 368, Pc Mean: 2.2097529699576683
Call PnL at 9:18 for strike: -3, Total: 807337.5000000002, Per Day: 2193.8519021739135, Days: 368, Pc Mean: 2.1964092790268843
Call PnL at 9:18 for strike: -2, Total: 720057.5, Per Day: 1956.6779891304348, Days: 368, Pc Mean: 1.9887079193200083
Call PnL at 9:18 for strike: -1, Total: 641890.0000000001, Per Day: 1744.2663043478265, Days: 368, Pc Mean: 1.8311249840868105
Call PnL at 9:18 for strike: 0, Total: 577307.5000000002, Per Day: 1568.7703804347832, Days: 368, Pc Mean: 1.619612769168954
Call PnL at 9:18 for strike: 1, Total: 526917.5, Per Day: 1455.5732044198894, Days: 362, Pc Mean: 1.5759072833721137
Call PnL at 9:18 for strike: 2, Total: 480467.50000000006, Per Day: 1392.6594202898552, Days: 345, Pc Mean: 1.5418813730866745
Call PnL at 9:18 for strike: 3, Total: 411579.99999999994, Per Day: 1331.9741100323622, Days: 309, Pc Mean: 1.5533763343764866
Call PnL at 9:18 for strike: 4, Total: 276877.5, Per Day: 1036.9943820224719, Days: 267, Pc Mean: 1.2849965646637946
Call PnL at 9:18 for strike: 5, Total: 149872.5000000001, Per Day: 724.0217391304352, Days: 207, Pc Mean: 0.7762541829311271
Call PnL at 9:18 for strike: 6, Total: 154740.0, Per Day: 979.367088607595, Days: 158, Pc Mean: 1.2739409931218943
Call PnL at 9:18 for strike: 7, Total: 10914.999999999995, Per Day: 94.09482758620685, Days: 116, Pc Mean: 0.12636272094800613
Call PnL at 9:18 for strike: 8, Total: 9917.499999999989, Per Day: 113.9942528735631, Days: 87, Pc Mean: 0.18813370356162376
Call PnL at 9:18 for strike: 9, Total: 79105.00000000006, Per Day: 1180.6716417910457, Days: 67, Pc Mean: 1.5818898689081564
Call PnL at 9:18 for strike: 10, Total: 35062.50000000002, Per Day: 637.5000000000003, Days: 55, Pc Mean: 1.1953436107573956
Call PnL at 9:18 for strike: 11, Total: 20739.99999999998, Per Day: 460.88888888888846, Days: 45, Pc Mean: 0.7593888029924587
Call PnL at 9:18 for strike: 12, Total: -55417.49999999998, Per Day: -1288.7790697674413, Days: 43, Pc Mean: -2.0308454534870015
Call PnL at 9:18 for strike: 13, Total: 21262.50000000002, Per Day: 574.6621621621626, Days: 37, Pc Mean: 0.8487115283167249
Call PnL at 9:18 for strike: 14, Total: -24174.999999999978, Per Day: -1510.9374999999986, Days: 16, Pc Mean: -1.9692430667294567

Results Training dataset 1st expiry
Call PnL at 9:18 for strike: -10, Total: -1200977.5, Per Day: -3263.5258152173915, Days: 368, Pc Mean: -3.2645265341374787
Call PnL at 9:18 for strike: -9, Total: -1498797.5, Per Day: -4072.819293478261, Days: 368, Pc Mean: -4.07595378510088
Call PnL at 9:18 for strike: -8, Total: -1580342.5, Per Day: -4294.408967391304, Days: 368, Pc Mean: -4.300811979583259
Call PnL at 9:18 for strike: -7, Total: -1589612.5, Per Day: -4319.599184782609, Days: 368, Pc Mean: -4.33081776982772
Call PnL at 9:18 for strike: -6, Total: -1313277.5, Per Day: -3568.688858695652, Days: 368, Pc Mean: -3.581478936364814
Call PnL at 9:18 for strike: -5, Total: -957462.4999999998, Per Day: -2601.8002717391296, Days: 368, Pc Mean: -2.6178756181139833
Call PnL at 9:18 for strike: -4, Total: -376049.9999999999, Per Day: -1021.8749999999997, Days: 368, Pc Mean: -1.0172680452979237
Call PnL at 9:18 for strike: -3, Total: 861522.5, Per Day: 2341.09375, Days: 368, Pc Mean: 2.3639857872910177
Call PnL at 9:18 for strike: -2, Total: 1831165.0, Per Day: 4975.991847826087, Days: 368, Pc Mean: 5.016709407810276
Call PnL at 9:18 for strike: -1, Total: 1928680.0, Per Day: 5240.978260869565, Days: 368, Pc Mean: 5.2977612436011565
Call PnL at 9:18 for strike: 0, Total: 1511580.0, Per Day: 4107.554347826087, Days: 368, Pc Mean: 4.227216010633293
Call PnL at 9:18 for strike: 1, Total: 1021664.9999999998, Per Day: 2776.263586956521, Days: 368, Pc Mean: 3.0202142490892543
Call PnL at 9:18 for strike: 2, Total: 790220.0000000002, Per Day: 2147.33695652174, Days: 368, Pc Mean: 2.2219541615482883
Call PnL at 9:18 for strike: 3, Total: 529924.9999999999, Per Day: 1443.9373297002721, Days: 367, Pc Mean: 1.659199832293209
Call PnL at 9:18 for strike: 4, Total: 404357.50000000006, Per Day: 1117.0096685082874, Days: 362, Pc Mean: 1.2296016828922414

Results Test dataset 1st expiry
Call PnL at 9:18 for strike: -5, Total: -368580.0000000001, Per Day: -996.1621621621625, Days: 370, Pc Mean: -0.9981795406298148
Call PnL at 9:18 for strike: -4, Total: -66219.99999999988, Per Day: -178.97297297297266, Days: 370, Pc Mean: -0.18476101433523964
Call PnL at 9:18 for strike: -3, Total: 29995.000000000087, Per Day: 81.0675675675678, Days: 370, Pc Mean: 0.07554203515727084
Call PnL at 9:18 for strike: -2, Total: 574157.5, Per Day: 1551.777027027027, Days: 370, Pc Mean: 1.5985776757810657
Call PnL at 9:18 for strike: -1, Total: 827162.5000000002, Per Day: 2235.574324324325, Days: 370, Pc Mean: 2.2925136232157
Call PnL at 9:18 for strike: 0, Total: 641042.5, Per Day: 1732.5472972972973, Days: 370, Pc Mean: 1.798247708632138
Call PnL at 9:18 for strike: 1, Total: 467930.0000000001, Per Day: 1264.675675675676, Days: 370, Pc Mean: 1.3283195796282112
Call PnL at 9:18 for strike: 2, Total: 390254.99999999994, Per Day: 1054.7432432432431, Days: 370, Pc Mean: 1.0203631153733186
Call PnL at 9:18 for strike: 3, Total: 209460.00000000003, Per Day: 567.6422764227643, Days: 369, Pc Mean: 0.7726631681100194
Call PnL at 9:18 for strike: 4, Total: 245740.00000000006, Per Day: 678.839779005525, Days: 362, Pc Mean: 0.7254304369662518

Results all dataset 1st expiry
Call PnL at 9:18 for strike: -3, Total: 891517.5, Per Day: 1208.0182926829268, Days: 738, Pc Mean: 1.2166630389312796
Call PnL at 9:18 for strike: -2, Total: 2405322.5, Per Day: 3259.2445799457996, Days: 738, Pc Mean: 3.303011926982623
Call PnL at 9:18 for strike: -1, Total: 2755842.5, Per Day: 3734.2039295392956, Days: 738, Pc Mean: 3.791065282161294
"""

# for i in range(-15, 30):
# for i in range(5, 10):
# for i in range(10, 15):
# for i in range(-10, -5):
# for i in range(-5, 0):
# for i in range(0, 5):
for i in range(0, 3):
    strike_key = f"cur_call_atm_strike_{i}"
    train_dates[strike_key] = train_dates["atm_strike"].apply(lambda r: get_next_1000(r, i))
    buy_at = f"call_at_0328_{i}"
    sell_at = f"call_at_0918_{i}"
    quantity_key = f"quantity_{i}"
    train_dates[buy_at] = train_dates.apply(lambda trade: get_premium_df(trade, trade[strike_key], trade.previous_trading_day.date(), OPTION_TYPE_CALL, dt.time(hour=15, minute=28)), axis=1)
    train_dates[sell_at] = train_dates.loc[train_dates[buy_at].notna()].apply(lambda trade: get_premium_df(trade, trade[strike_key], trade.name.date(), OPTION_TYPE_CALL, dt.time(hour=9, minute=18)), axis=1)
    train_dates[quantity_key] = train_dates.loc[train_dates[buy_at].notna()][buy_at].apply(lambda buy_price: ut.get_quantity(buy_price=buy_price, lot_size=LOT_SIZE, investment=DAILY_INVESTMENT))
    pnl_key = f"call_pnl_0918_{i}"
    train_dates[pnl_key] = train_dates.loc[train_dates[buy_at].notna()][quantity_key] * (train_dates.loc[train_dates[buy_at].notna()][sell_at] - train_dates.loc[train_dates[buy_at].notna()][buy_at])
    pnl_pc_key = f"call_pnl_pc_{i}"
    train_dates[pnl_pc_key] = train_dates.loc[train_dates[buy_at].notna()][pnl_key] * 100 / (train_dates.loc[train_dates[buy_at].notna()][quantity_key] * train_dates.loc[train_dates[buy_at].notna()][buy_at])
    print(f"Call PnL at 9:18 for strike: {i}, Total: {train_dates[pnl_key].sum()}, Per Day: {train_dates[pnl_key].mean()}, Days: {train_dates.loc[train_dates[pnl_key].notna()].shape[0]}, Pc Mean: {train_dates[pnl_pc_key].mean()}")


Call PnL at 9:18 for strike: 0, Total: 2152622.5, Per Day: 2916.832655826558, Days: 738, Pc Mean: 3.0094405746706547
Call PnL at 9:18 for strike: 1, Total: 1489595.0, Per Day: 2018.4214092140921, Days: 738, Pc Mean: 2.1719743741562105
Call PnL at 9:18 for strike: 2, Total: 1180475.0, Per Day: 1599.559620596206, Days: 738, Pc Mean: 1.6195304663115149


In [6]:
num = -1
pd.set_option("display.max_colwidth", None)
pd.set_option("display.max_rows", 2000)
train_dates["dd"] = train_dates[f"call_pnl_0918_{num}"]

for idx, row in train_dates.iterrows():
    pr = train_dates.shift(1).loc[idx]
    if pd.isna(pr.atm_strike) or pd.isnull(pr.atm_strike):
        continue
    train_dates.loc[idx, "dd"] = pr.dd + row[f"call_pnl_0918_{num}"]

def format_float(value):
    return f'{value:.2f}'

# Set the display.float_format option to use the formatting function
pd.options.display.float_format = format_float

train_dates[["previous_trading_day", "expiry", "nifty_at_0320", "atm_strike", f"cur_call_atm_strike_{num}", f"call_at_0328_{num}", f"call_at_0918_{num}", f"call_pnl_0918_{num}", "nifty_diff", "nifty_at_buy", "nifty_at_sell", f"call_pnl_pc_{num}", f"quantity_{num}", "dd"]]

,previous_trading_day,expiry,nifty_at_0320,atm_strike,cur_call_atm_strike_-1,call_at_0328_-1,call_at_0918_-1,call_pnl_0918_-1,nifty_diff,nifty_at_buy,nifty_at_sell,call_pnl_pc_-1,quantity_-1,dd
trade_date,,,,,,,,,,,,,,
2021-01-01,2020-12-31,2021-01-07,13979.90,14000,14000,123.30,129.85,5240.00,33.45,13980.20,14013.65,5.31,800.00,5240.00
2021-01-04,2021-01-01,2021-01-07,14013.60,14000,14000,122.20,149.35,21720.00,65.40,14017.85,14083.25,22.22,800.00,26960.00
2021-01-05,2021-01-04,2021-01-07,14129.55,14150,14200,55.65,36.95,-32725.00,-52.30,14143.75,14091.45,-33.60,1750.00,-5765.00
2021-01-06,2021-01-05,2021-01-07,14207.60,14200,14200,69.55,75.60,8470.00,21.35,14206.95,14228.30,8.70,1400.00,2705.00
2021-01-07,2021-01-06,2021-01-07,14147.60,14150,14200,32.40,55.35,69997.50,90.05,14134.85,14224.90,70.83,3050.00,72702.50
2021-01-08,2021-01-07,2021-01-14,14130.15,14150,14200,113.00,131.55,15767.50,108.75,14137.35,14246.10,16.42,850.00,88470.00
2021-01-11,2021-01-08,2021-01-14,14352.45,14350,14400,81.35,104.85,28200.00,55.70,14354.90,14410.60,28.89,1200.00,116670.00
2021-01-12,2021-01-11,2021-01-14,14483.75,14500,14500,79.70,64.80,-18625.00,-31.45,14489.30,14457.85,-18.70,1250.00,98045.00
2021-01-13,2021-01-12,2021-01-14,14559.50,14550,14600,59.50,68.85,15427.50,57.60,14574.00,14631.60,15.71,1650.00,113472.50


In [9]:
"""
Results all_dates i=-1 expiry 1st

Month: 2021-01-31 00:00:00, profit: 139605.0
Month: 2021-02-28 00:00:00, profit: 71115.0
Month: 2021-03-31 00:00:00, profit: 16797.5
Month: 2021-04-30 00:00:00, profit: 126210.0
Month: 2021-05-31 00:00:00, profit: 34687.5
Month: 2021-06-30 00:00:00, profit: 47577.5
Month: 2021-07-31 00:00:00, profit: 8377.5
Month: 2021-08-31 00:00:00, profit: 432315.0
Month: 2021-09-30 00:00:00, profit: 216330.0
Month: 2021-10-31 00:00:00, profit: 334595.0
Month: 2021-11-30 00:00:00, profit: -63745.0
Month: 2021-12-31 00:00:00, profit: 391830.0
Month: 2022-01-31 00:00:00, profit: -192085.0
Month: 2022-02-28 00:00:00, profit: -70482.5
Month: 2022-03-31 00:00:00, profit: 92437.5
Month: 2022-04-30 00:00:00, profit: -108052.5
Month: 2022-05-31 00:00:00, profit: -42137.5
Month: 2022-06-30 00:00:00, profit: -240587.5
Month: 2022-07-31 00:00:00, profit: 418250.0
Month: 2022-08-31 00:00:00, profit: 83580.0
Month: 2022-09-30 00:00:00, profit: -42612.5
Month: 2022-10-31 00:00:00, profit: 324885.0
Month: 2022-11-30 00:00:00, profit: -49162.5
Month: 2022-12-31 00:00:00, profit: -125740.0
Month: 2023-01-31 00:00:00, profit: -59820.0
Month: 2023-02-28 00:00:00, profit: -134817.5
Month: 2023-03-31 00:00:00, profit: 156280.0
Month: 2023-04-30 00:00:00, profit: -48010.0
Month: 2023-05-31 00:00:00, profit: 188342.5
Month: 2023-06-30 00:00:00, profit: 104335.0
Month: 2023-07-31 00:00:00, profit: 315392.5
Month: 2023-08-31 00:00:00, profit: -118622.5
Month: 2023-09-30 00:00:00, profit: -151150.0
Month: 2023-10-31 00:00:00, profit: -229855.0
Month: 2023-11-30 00:00:00, profit: 330382.5
Month: 2023-12-31 00:00:00, profit: 599397.5
Year: 2021-12-31 00:00:00, profit: 1755695.0
Year: 2022-12-31 00:00:00, profit: 48292.5
Year: 2023-12-31 00:00:00, profit: 951855.0
"""
pnl_key = f"call_pnl_0918_{num}"
train_dates["trade_date"] = train_dates.index.values
gdf = train_dates.groupby(pd.Grouper(key="trade_date", freq="ME"))
for month, mdf in gdf:
    print(f"Month: {month}, profit: {round(mdf[pnl_key].sum(), 2)}")

gdf = train_dates.groupby(pd.Grouper(key="trade_date", freq="YE"))
for year, mdf in gdf:
    print(f"Year: {year}, profit: {round(mdf[pnl_key].sum(), 2)}")


Month: 2021-01-31 00:00:00, profit: 139605.0
Month: 2021-02-28 00:00:00, profit: 71115.0
Month: 2021-03-31 00:00:00, profit: 16797.5
Month: 2021-04-30 00:00:00, profit: 126210.0
Month: 2021-05-31 00:00:00, profit: 34687.5
Month: 2021-06-30 00:00:00, profit: 47577.5
Month: 2021-07-31 00:00:00, profit: 8377.5
Month: 2021-08-31 00:00:00, profit: 432315.0
Month: 2021-09-30 00:00:00, profit: 216330.0
Month: 2021-10-31 00:00:00, profit: 334595.0
Month: 2021-11-30 00:00:00, profit: -63745.0
Month: 2021-12-31 00:00:00, profit: 391830.0
Month: 2022-01-31 00:00:00, profit: -192085.0
Month: 2022-02-28 00:00:00, profit: -70482.5
Month: 2022-03-31 00:00:00, profit: 92437.5
Month: 2022-04-30 00:00:00, profit: -108052.5
Month: 2022-05-31 00:00:00, profit: -42137.5
Month: 2022-06-30 00:00:00, profit: -240587.5
Month: 2022-07-31 00:00:00, profit: 418250.0
Month: 2022-08-31 00:00:00, profit: 83580.0
Month: 2022-09-30 00:00:00, profit: -42612.5
Month: 2022-10-31 00:00:00, profit: 324885.0
Month: 2022-11-

In [7]:
"""
Training dataset 1st expiry, i=0
-261950.0
2021-03-31 00:00:00

Test dataset 1st expiry i=0
18250.0
2021-01-04 00:00:00

All dataset 1st expiry i=-1:
-5765.0
2021-01-05 00:00:00
"""

print(train_dates["dd"].min())
print(train_dates["dd"].idxmin())

-5765.0
2021-01-05 00:00:00


In [11]:
train_dates = train_dates.to_csv("gapup_results.csv")
# train_dates = pd.read_csv("gapup_results.csv")